In [1]:
import requests
import re
import json5
import pandas as pd
from tqdm.contrib import itertools
import csv
import numpy as np

In [2]:
mss_url="http://www.weather.gov.sg/climate-historical-daily/"
mss_csv="http://www.weather.gov.sg/files/dailydata/DAILYDATA_{}_{}{:02d}.csv"

In [3]:
#retrieve HTML code and remove linebreaks
mss=requests.get(mss_url)
mss_text=mss.text
mss_text = mss_text.replace("\n","").replace("\r","").replace("\t","")

In [4]:
#use regex to capture the weather stations, their names, and the station codes
regex=r"stations\[\d+\]=({[^\}]*})"
mss_list=re.findall(regex,mss_text)

In [5]:
#convert the captured javascript object to python dictionary
mss_list_of_dict=[]
for i in range(len(mss_list)):
    mss_list_of_dict.append(json5.loads(mss_list[i]))

In [6]:
#convert dictionary of stations to dataframe
df_stn=pd.DataFrame(mss_list_of_dict)

In [7]:
#get list of stations
list_of_station_codes=df_stn['station_code'].to_list()

In [8]:
#set starting and ending year/month
y_start=2009
y_end=2022
m_start=1
m_end=12

Although multithreading is faster here, but we'll probably DDOS the server if we do that.  
So let's keep to single thread.

In [9]:
columns=["Station","Year","Month","Day","Daily Rainfall Total",\
         "Highest 30 Min Rainfall","Highest 60 Min Rainfall","Highest 120 Min Rainfall",\
         "Mean Temperature","Maximum Temperature","Minimum Temperature","Mean Wind Speed","Max Wind Speed"]

In [10]:
%%time
#get all files from server, check for status code = 200, store to list
csv_list=[]
for stn, y, m in itertools.product(list_of_station_codes,range(y_start,y_end+1),range(m_start,m_end+1)):
    url=mss_csv.format(stn,y,m)
    r = requests.get(url)  
    if r.status_code==200:
        decoded_content = r.content.decode('ISO-8859-1')
        current_file = list(csv.reader(decoded_content.splitlines()))
        current_file.pop(0) #remove header
        csv_list.extend(current_file)

  0%|          | 0/10584 [00:00<?, ?it/s]

CPU times: total: 52.9 s
Wall time: 15min 30s


In [19]:
#create dataframe of mss data
df_mss=pd.DataFrame(csv_list,columns=columns)
df_mss=df_mss.replace('-', np.nan)
df_mss=df_mss.replace('', np.nan)

In [20]:
df_mss

,Station,Year,Month,Day,Daily Rainfall Total,Highest 30 Min Rainfall,Highest 60 Min Rainfall,Highest 120 Min Rainfall,Mean Temperature,Maximum Temperature,Minimum Temperature,Mean Wind Speed,Max Wind Speed
0,Paya Lebar,2009,1,1,0,NaN,NaN,NaN,NaN,32.7,24,8.4,NaN
1,Paya Lebar,2009,1,2,0.5,NaN,NaN,NaN,NaN,30.5,24,12.3,NaN
2,Paya Lebar,2009,1,3,0,NaN,NaN,NaN,NaN,30.1,25,13.5,NaN
3,Paya Lebar,2009,1,4,1.6,NaN,NaN,NaN,NaN,30.2,24.6,13,NaN
4,Paya Lebar,2009,1,5,2.5,NaN,NaN,NaN,NaN,32.3,25.2,15.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283902,Pasir Ris (Central),2022,10,27,0.8,0.4,0.6,0.8,NaN,NaN,NaN,NaN,NaN
283903,Pasir Ris (Central),2022,10,28,4.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN
283904,Pasir Ris (Central),2022,10,29,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
283905,Pasir Ris (Central),2022,10,30,39.2,24.8,26.2,39.2,NaN,NaN,NaN,NaN,NaN


In [23]:
df_mss.to_csv('../assets/mss/mss_data.csv')

In [14]:
df_stn.to_csv('../assets/mss/stations.csv')